In [ ]:
# from langchain.llms.openai import OpenAI # text-davinci-003 Model (사용 안 함)
from langchain.chat_models import ChatOpenAI # GPT-3.5-turbo Model (text-davinci-003의 1/10 가격)

# llm = OpenAI()
chat = ChatOpenAI(
    temperature=0.1 # 낮을 수록 일관성 있음, 높을 수록 무작위성(창의성) 있음, default 0.7
)

In [ ]:
##### 방식 1. HumanMessage, AIMessage, SystemMessage 사용 #####
from langchain.schema import HumanMessage, AIMessage, SystemMessage # HumanMessage: 사용자 입력, AIMessage: AI 출력, SystemMessage: 시스템 출력, prompt를 직접 입력할 때 사용

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in {language}."),
    AIMessage(content="Ciao, mi chiamo {name}!"),
    HumanMessage(content="What is the distance between {country_a} and {country_b}? Also, what is your name?")
]

chat.predict_messages(messages)

In [ ]:
##### 방식 2. PromptTemplate 사용 #####
from langchain.prompts import PromptTemplate # PromtTemplate은 string에서 template을 생성하기 위해 사용

template_c = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}? Also, what is your name?") # template 생성, {country_a}, {country_b}는 변수

prompt_a = template_c.format(country_a="Maxico", country_b="Thailand") # template에 변수를 적용하여 prompt 생성

chat.predict(prompt_a) # prompt_a를 입력하여 AI 출력 예측

In [ ]:
##### 방식 3. ChatPromptTemplate 사용 #####
from langchain.prompts import ChatPromptTemplate # CharPromptTemplate는 messages에서 template을 생성하기 위해 사용

template_b = ChatPromptTemplate.from_messages( # system, ai, human 순서로 messages를 ("speaker", "content") 형태로 입력, {language}, {name}, {country_a}, {country_b}는 변수
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Ciao, mi chiamo {name}!"),
        ("human", "What is the distance between {country_a} and {country_b}? Also, what is your name?")
    ]
)

prompt_b = template_b.format_messages( # template에 변수를 적용하여 prompt 생성
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand"
)

chat.predict_messages(prompt_b) # prompt_b를 입력하여 AI 출력 예측

In [ ]:
##### 방식 4. ChatPromptTemplate 사용 및 사용자 정의 함수를 사용한 응용 #####
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser): # AI 출력을 파싱하기 위한 클래스, BaseOutputParser를 상속받아 구현, 파싱이란? AI 출력을 의미있는 형태로 변환하는 것
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

par = CommaOutputParser() # CommaOutputParser 객체 par 생성 (함수를 사용하기 위해 생성 필요함)

template_c = ChatPromptTemplate.from_messages( # {max_items}은 답변의 최대 개수, {question}은 사용자의 질문
    [
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lower case. Do NOT reply with anything else."),
        ("human", "{question}")
    ]
)

prompt_c = template_c.format_messages( # template에 변수를 적용하여 prompt 생성
    max_items=10,
    question="What are the planets?"
)

result = chat.predict_messages(prompt_c) # prompt_c를 입력하여 AI 출력 예측

par.parse(result.content)

In [ ]:
##### 방식 5. Langchain format을 사용한 응용 #####

chain = template_c | chat | par # template -> chat -> par 순서로 실행됨

chain.invoke(
    {
        "max_items": 10,
        "question": "What are the pokemons?"
    }
)

# 이런 방식으로도 조합할 수 있음
# chain_one = template1 | chat1 | parser1: template1 -> chat1 -> parser1 순서로 실행됨
# chain_two = template2 | chat2 | parser2: template2 -> chat2 -> parser2 순서로 실행됨
# all = chain_one | chain_two | ouput: chain_one -> chain_two -> output 순서로 실행됨

* LangChain?
<br/> template1 | chat1 | parser1 로 짜여진 chain을 invoke 할 경우
<br/> 첫 번째 input이 template1로 입력되어 첫 번째 output이 되어 결과로 나오고
<br/> 해당 output1이 chat1로 입력되어 두 번째 output이 되어 결과로 나오고
<br/> 해당 output2가 parser1로 입력되어 최종 결과로 나옴 

5 Input Types of LangChain
1. Prompt: Dict Type
2. Retriever: String Type
3. LLM: String Type
    3-1. ChatModel: List Type (chat message나 PromptValue Type 자료를 담고 있음)
4. Tool: String Type
5. OutputParser

Output Types of LangChain
1. LLM: String Type
    1-1. ChatModel: ChatMessage Type
2. Prompt: PromptValue Type
3. Retriever: List of Documents
4. Tool: Depends on the tool
5. OutputParser: Depends on the parser

---

In [3]:
# from langchain.llms.openai import OpenAI # text-davinci-003 Model (사용 안 함)
from langchain.chat_models import ChatOpenAI # GPT-3.5-turbo Model (text-davinci-003의 1/10 가격)
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# llm = OpenAI()
chat = ChatOpenAI(
    temperature=0.1 # 낮을 수록 일관성 있음, 높을 수록 무작위성(창의성) 있음, default 0.7
    ,streaming=True # streaming=True로 설정하면, AI 출력이 나올 때마다 출력을 반환함, 모든 출력이 다 나올때까지 대기할 필요가 없음
    ,callbacks=[StreamingStdOutCallbackHandler()] # streaming=True로 설정하면, 출력을 받을 때마다 callback 함수를 호출함
)

In [4]:
# {요리 이름}을 입력하면 요리법을 알려주는 챗봇
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipe for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat

In [5]:
# {요리법}을 입력하면 해당 요리법의 재료를 Vegetarian 용으로 바꿔주는 챗봇
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}"),
])
# "If there is no alternative for a food just say you don't know how to replace it." Hallucination 방지를 위한 문장

In [6]:
veg_chain = veg_chef_prompt | chat

# chain model을 연계하여 사용하지 않을 경우 chef_chain, veg_chain을 각각 사용해야 함
# chef_chain.invoke({"cuisine": "indian"}) -> "Chicken Tikka Masala"
# veg_chain.invoke({"recipe": "Chicken Tikka Masala"}) -> veg_recipe of "Chicken Tikka Masala"

# final_chain = chef_chain | veg_chain

# 반면 chain model을 연계하여 사용할 경우 chef_chain, veg_chain을 각각 사용하지 않아도 됨

final_chain = {"recipe": chef_chain} | veg_chain # chef_chain의 결과값이 veg_chain의 {recipe}에 들어감
final_chain.invoke({
    "cuisine": "indian" #cuisine이 chef_chain의 {cuisine}에 들어감
})

Great choice! Indian cuisine is known for its rich flavors and aromatic spices. Let's start with a classic Indian dish called Butter Chicken. Here's an easy recipe for you:

Ingredients:
- 500g boneless chicken, cut into bite-sized pieces
- 2 tablespoons butter
- 1 onion, finely chopped
- 2 cloves of garlic, minced
- 1-inch piece of ginger, grated
- 2 teaspoons garam masala
- 1 teaspoon turmeric powder
- 1 teaspoon chili powder (adjust to your spice preference)
- 1 cup tomato puree
- 1/2 cup heavy cream
- Salt to taste
- Fresh cilantro leaves for garnish

Instructions:
1. Heat the butter in a large pan over medium heat. Add the chopped onion and sauté until it turns golden brown.
2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.
3. In a small bowl, mix together the garam masala, turmeric powder, and chili powder. Add this spice mixture to the pan and cook for a minute to release the flavors.
4. Add the chicken pieces to the pan and cook unti

AIMessageChunk(content="Great choice! Butter Chicken is a delicious Indian dish that can easily be made vegetarian. Here's a modified version of the recipe using alternative ingredients:\n\nIngredients:\n- 500g paneer (Indian cottage cheese), cut into bite-sized pieces\n- 2 tablespoons butter (or vegan butter for a vegan option)\n- 1 onion, finely chopped\n- 2 cloves of garlic, minced\n- 1-inch piece of ginger, grated\n- 2 teaspoons garam masala\n- 1 teaspoon turmeric powder\n- 1 teaspoon chili powder (adjust to your spice preference)\n- 1 cup tomato puree\n- 1/2 cup coconut cream (or cashew cream for a nutty flavor)\n- Salt to taste\n- Fresh cilantro leaves for garnish\n\nInstructions:\n1. Heat the butter in a large pan over medium heat. Add the chopped onion and sauté until it turns golden brown.\n2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.\n3. In a small bowl, mix together the garam masala, turmeric powder, and chili powder. Add thi

위 코드를 실행한 결과

AIMessage(
    content=
    "Great choice! Butter Chicken is a delicious Indian dish. 
    To make it vegetarian, we can replace the chicken with a plant-based alternative. 
    Here's an alternative recipe for Vegetarian Butter Chicken:\n\n
    
    Ingredients:\n
    - 500g plant-based chicken substitute (such as tofu, tempeh, or seitan), cut into bite-sized pieces\n
    - 2 tablespoons butter or vegan butter substitute\n
    - 1 onion, finely chopped\n- 2 cloves of garlic, minced\n
    - 1-inch piece of ginger, grated\n- 2 teaspoons garam masala\n
    - 1 teaspoon turmeric powder\n
    - 1 teaspoon chili powder (adjust according to your spice preference)\n
    - 1 cup tomato puree\n- 1/2 cup coconut cream or cashew cream (for a creamy texture)\n
    - Salt to taste\n
    - Fresh cilantro leaves, for garnish\n\n
    
    Instructions:\n
    1. Heat the butter in a large pan over medium heat. Add the chopped onions and sauté until they turn golden brown.\n
    2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.\n
    3. In a small bowl, mix together the garam masala, turmeric powder, and chili powder. Add this spice mixture to the pan and cook for a minute to release the flavors.\n
    4. Add the plant-based chicken substitute to the pan and cook until it is lightly browned on all sides.\n
    5. Pour in the tomato puree and season with salt. Stir well to combine all the ingredients.\n
    6. Reduce the heat to low, cover the pan, and let the mixture simmer for about 15-20 minutes, or until the plant-based chicken substitute is cooked through and tender.\n
    7. Stir in the coconut cream or cashew cream and simmer for an additional 5 minutes to thicken the sauce.\n
    8. Taste and adjust the seasoning if needed.\n
    9. Garnish with fresh cilantro leaves before serving.\n
    10. Serve the Vegetarian Butter Chicken hot with steamed rice or naan bread.\n\n
    
     Enjoy your homemade Vegetarian Butter Chicken! You can also add vegetables like bell peppers, peas, or mushrooms to enhance the dish."
)

Great choice! Indian cuisine is known for its rich flavors and aromatic spices. Let's start with a classic Indian dish called Butter Chicken. Here's an easy recipe for you:

Ingredients:
- 500g boneless chicken, cut into bite-sized pieces
- 2 tablespoons butter
- 1 onion, finely chopped
- 2 cloves of garlic, minced
- 1-inch piece of ginger, grated
- 2 teaspoons garam masala
- 1 teaspoon turmeric powder
- 1 teaspoon chili powder (adjust to your spice preference)
- 1 cup tomato puree
- 1/2 cup heavy cream
- Salt to taste
- Fresh cilantro leaves for garnish

Instructions:
1. Heat the butter in a large pan over medium heat. Add the chopped onion and sauté until it turns golden brown.
2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.
3. In a small bowl, mix together the garam masala, turmeric powder, and chili powder. Add this spice mixture to the pan and cook for a minute to release the flavors.
4. Add the chicken pieces to the pan and cook until they are browned on all sides.
5. Pour in the tomato puree and stir well to combine. Reduce the heat to low, cover the pan, and let it simmer for about 15-20 minutes, or until the chicken is cooked through.
6. Stir in the heavy cream and season with salt to taste. Simmer for an additional 5 minutes to allow the flavors to meld together.
7. Garnish with fresh cilantro leaves before serving.
8. Serve the Butter Chicken with steamed basmati rice or naan bread for a complete meal.
...
6. Stir in the coconut cream (or cashew cream) and season with salt to taste. Simmer for an additional 5 minutes to allow the flavors to meld together.
7. Garnish with fresh cilantro leaves before serving.
8. Serve the Butter Paneer with steamed basmati rice or naan bread for a complete meal.

Enjoy your homemade vegetarian Butter Chicken!

---